Second Cell in Notebook - JOINS, Dataset Used - COVID 19 Dataset

In [2]:
##JOINS
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CovidAnalysis").getOrCreate()
country_wise_df = spark.read.csv("D:/BL_DI_DE/Database Problem/data/country_wise_latest.csv", header=True, inferSchema=True)

In [3]:
##setting the csv's as dataframes
country_df = spark.read.option("header", True).option("inferSchema", True).csv("country_wise_latest.csv")
clean_df = spark.read.option("header", True).option("inferSchema", True).csv("covid_19_clean_complete.csv")

In [6]:
country_df.show(6)

+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
|     Country/Region|Confirmed|Deaths|Recovered|Active|New cases|New deaths|New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|Confirmed last week|1 week change|1 week % increase|          WHO Region|
+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
|        Afghanistan|    36263|  1269|    25198|  9796|      106|        10|           18|               3.5|                69.49|                  5.04|              35526|          737|             2.07|Eastern Mediterra...|
|            Albania|     4880|   144|     2745|  1991|      117|         6|           6

In [7]:
clean_df.show(5)

+--------------+--------------+--------+---------+----------+---------+------+---------+------+--------------------+
|Province/State|Country/Region|     Lat|     Long|      Date|Confirmed|Deaths|Recovered|Active|          WHO Region|
+--------------+--------------+--------+---------+----------+---------+------+---------+------+--------------------+
|          NULL|   Afghanistan|33.93911|67.709953|2020-01-22|        0|     0|        0|     0|Eastern Mediterra...|
|          NULL|       Albania| 41.1533|  20.1683|2020-01-22|        0|     0|        0|     0|              Europe|
|          NULL|       Algeria| 28.0339|   1.6596|2020-01-22|        0|     0|        0|     0|              Africa|
|          NULL|       Andorra| 42.5063|   1.5218|2020-01-22|        0|     0|        0|     0|              Europe|
|          NULL|        Angola|-11.2027|  17.8739|2020-01-22|        0|     0|        0|     0|              Africa|
+--------------+--------------+--------+---------+----------+---

In [8]:
joined_df = clean_df.join(country_df, on="Country/Region", how="inner").show(5)

+--------------+--------------+--------+---------+----------+---------+------+---------+------+--------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
|Country/Region|Province/State|     Lat|     Long|      Date|Confirmed|Deaths|Recovered|Active|          WHO Region|Confirmed|Deaths|Recovered|Active|New cases|New deaths|New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|Confirmed last week|1 week change|1 week % increase|          WHO Region|
+--------------+--------------+--------+---------+----------+---------+------+---------+------+--------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
|   Afghanistan|          NULL|

In [10]:
##Joining two dataframes as it is a natural join since both files contain Country/Region
worldometer_df = spark.read.csv("D:/BL_DI_DE/Database Problem/data/worldometer_data.csv", header=True, inferSchema=True)
join_worldometer_df = country_df.join(
    worldometer_df,
    country_df["Country/Region"] == worldometer_df["Country/Region"],
    "inner"
)
result_df = join_worldometer_df.select(
    country_df["`Country/Region`"],
    worldometer_df["`Country/Region`"]
)    
result_df.show()

+--------------+--------------+
|Country/Region|Country/Region|
+--------------+--------------+
|        Brazil|        Brazil|
|         India|         India|
|        Russia|        Russia|
|  South Africa|  South Africa|
|        Mexico|        Mexico|
|          Peru|          Peru|
|         Chile|         Chile|
|      Colombia|      Colombia|
|         Spain|         Spain|
|          Iran|          Iran|
|  Saudi Arabia|  Saudi Arabia|
|      Pakistan|      Pakistan|
|    Bangladesh|    Bangladesh|
|         Italy|         Italy|
|        Turkey|        Turkey|
|     Argentina|     Argentina|
|       Germany|       Germany|
|        France|        France|
|          Iraq|          Iraq|
|   Philippines|   Philippines|
+--------------+--------------+
only showing top 20 rows



In [12]:
##joining DATE column of both covid_19_clean_complete & day_wise
day_wise_df = spark.read.csv("D:/BL_DI_DE/Database Problem/data/day_wise.csv", header=True, inferSchema=True)

join_daywise_df = clean_df.join(
    day_wise_df,
    clean_df["Date"] == day_wise_df["Date"],
    "inner"
)

final_df = join_daywise_df.select(
    clean_df["Date"],
    day_wise_df["Date"]
)
join_daywise_df.show(5)

+--------------+--------------+--------+---------+----------+---------+------+---------+------+--------------------+----------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+----------------+
|Province/State|Country/Region|     Lat|     Long|      Date|Confirmed|Deaths|Recovered|Active|          WHO Region|      Date|Confirmed|Deaths|Recovered|Active|New cases|New deaths|New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|No. of countries|
+--------------+--------------+--------+---------+----------+---------+------+---------+------+--------------------+----------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+----------------+
|          NULL|   Afghanistan|33.93911|67.709953|2020-01-22|        0|     0|        0|     0|Eastern Mediterra...|2020-01-22|      555|    17|       28|   510|     

In [13]:
type(day_wise_df)

pyspark.sql.dataframe.DataFrame

In [15]:
##INNER JOIN to get region-wise confirmed cases with population
print("country_df:", country_df.columns)
print("clean_df:", clean_df.columns)

joined_df = country_df.join(
    clean_df,
    country_df["`Country/Region`"] == clean_df["`Country/Region`"],
    "inner"
)

result_df = joined_df.select(
    country_df["`Country/Region`"].alias("Country"),
    country_df["Confirmed"].alias("Total_Confirmed"),
    clean_df["`WHO Region`"].alias("WHO_Region")
)

result_df.show(10, truncate=False)

country_df: ['Country/Region', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'New cases', 'New deaths', 'New recovered', 'Deaths / 100 Cases', 'Recovered / 100 Cases', 'Deaths / 100 Recovered', 'Confirmed last week', '1 week change', '1 week % increase', 'WHO Region']
clean_df: ['Province/State', 'Country/Region', 'Lat', 'Long', 'Date', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'WHO Region']
+-------------------+---------------+---------------------+
|Country            |Total_Confirmed|WHO_Region           |
+-------------------+---------------+---------------------+
|Afghanistan        |36263          |Eastern Mediterranean|
|Albania            |4880           |Europe               |
|Algeria            |27973          |Africa               |
|Andorra            |907            |Europe               |
|Angola             |950            |Africa               |
|Antigua and Barbuda|86             |Americas             |
|Argentina          |167416         |Americas             |
|

In [26]:
#Data of all current country-level stats even if clean_df has missing entries
left_join_df = country_df.join(
    clean_df,
    country_df["`Country/Region`"] == clean_df["`Country/Region`"],
    "left"
)

structured_df = left_join_df.select(
    country_df["`Country/Region`"].alias("Country"),
    clean_df["Date"],
    clean_df["Confirmed"],
    clean_df["Deaths"],
    clean_df["Recovered"],
    country_df["Confirmed"].alias("Total_Confirmed"),
    country_df["Deaths"].alias("Total_Deaths"),
    country_df["`WHO Region`"].alias("WHO_Region")
)

structured_df.show(10, truncate=False)


+-----------+----------+---------+------+---------+---------------+------------+---------------------+
|Country    |Date      |Confirmed|Deaths|Recovered|Total_Confirmed|Total_Deaths|WHO_Region           |
+-----------+----------+---------+------+---------+---------------+------------+---------------------+
|Afghanistan|2020-07-27|36263    |1269  |25198    |36263          |1269        |Eastern Mediterranean|
|Afghanistan|2020-07-26|36157    |1259  |25180    |36263          |1269        |Eastern Mediterranean|
|Afghanistan|2020-07-25|36036    |1248  |24793    |36263          |1269        |Eastern Mediterranean|
|Afghanistan|2020-07-24|35981    |1225  |24602    |36263          |1269        |Eastern Mediterranean|
|Afghanistan|2020-07-23|35928    |1211  |24550    |36263          |1269        |Eastern Mediterranean|
|Afghanistan|2020-07-22|35727    |1190  |23924    |36263          |1269        |Eastern Mediterranean|
|Afghanistan|2020-07-21|35615    |1186  |23741    |36263          |1269  

In [25]:
#FILTER after JOIN
filtered_df = country_df.join(
    clean_df,
    country_df["`Country/Region`"] == clean_df["`Country/Region`"],
    "inner"
).filter(
    (country_df["`Country/Region`"] == "India") &
    (clean_df["Date"] == "2020-06-01")
)

result_df = filtered_df.select(
    country_df["`Country/Region`"].alias("Country"),
    clean_df["Date"],
    clean_df["Confirmed"],
    clean_df["Deaths"],
    clean_df["Recovered"],
    country_df["`WHO Region`"].alias("WHO_Region")
)

result_df.show(truncate=False)

+-------+----------+---------+------+---------+---------------+
|Country|Date      |Confirmed|Deaths|Recovered|WHO_Region     |
+-------+----------+---------+------+---------+---------------+
|India  |2020-06-01|198370   |5608  |95754    |South-East Asia|
+-------+----------+---------+------+---------+---------------+

